In [2]:
import pandas as pd
import json
import logging
import time
import backoff
import boto3
import pandas as pd


logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()

pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 1000)

client = boto3.client("lambda")


df = pd.read_csv(
    filepath_or_buffer="../data/enriched/votos_por_secao_com_cep.csv",
    encoding="utf-8",
    header=0,
    dtype={
        0: int,
        1: int,
        2: str,
        3: str,
        4: str,
        5: str,
        6: str,
        7: int,
        8: int,
        9: str,
        10: int,
        },
        nrows=10000
    )
df["CEP"] = df["CEP"].str[:-4]

df = df[["CEP", "TURNO", "NUMERO_CANDIDATO", "VOTOS"]]
df = df.groupby(["CEP", "TURNO", "NUMERO_CANDIDATO"]).sum()
df = df.reset_index()
total_rows = df.shape[0]

@backoff.on_exception(backoff.expo, RuntimeError, max_time=15)
def call_lambda(payload):
    print(payload)
    response = client.invoke(
        FunctionName="Eleicoes2022-DataProcessi-GeoVotesFunctionCA5674F9-5OLJWl1KhVGr",
        InvocationType="Event",
        Payload=json.dumps(payload),
    )
    status_code = response['StatusCode']
    print(status_code)
    if status_code < 200 and status_code >= 300:
        raise RuntimeError

for row in df.itertuples():
    print(f"Processing row {row.Index} of {total_rows}")
    payload = {"cep": row.CEP, "turno": row.TURNO, "numero_candidato": row.NUMERO_CANDIDATO, "votos": row.VOTOS}
    call_lambda(payload)
    break


Processing row 0 of 274
{'cep': '0957', 'numero_candidato': 12, 'votos': 276}
202
